In [34]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import EnvironmentSetup as es

In [35]:
es.setupEnvironment()
import ConnectionConfig as cc
spark = cc.startLocalCluster("Kafka")
spark.getActiveSession()
wordcount_delta_path = "./spark-warehouse/kafkaWordCount"

In [36]:
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "127.0.0.1:29092")
    .option("subscribe", "demo")
    .load()
)

df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [37]:
from pyspark.sql import functions as F

#df.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")
inter = df.selectExpr("json_tuple(CAST(value as STRING), 'event_time', 'word', 'count')")
inter.printSchema()
#stream = inter.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")

root
 |-- c0: string (nullable = true)
 |-- c1: string (nullable = true)
 |-- c2: string (nullable = true)



In [41]:
wordCountEvents= inter.selectExpr("CAST(c0 as timestamp) as timestamp" , "c1 as key", "INT(c2) as value").withWatermark("timestamp",'10 seconds')
wordCountEvents.printSchema()
#stream = wordCountEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")


root
 |-- timestamp: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- value: integer (nullable = true)



In [46]:
groupedEvents =wordCountEvents.groupBy(F.window("timestamp", "10 seconds", "10 seconds"), "key").sum("value").withColumnRenamed("sum(value)", "count")
groupedEvents.printSchema()
stream = groupedEvents.writeStream.format("delta").outputMode("append").option("checkpointLocation",".\checkpoints").option("path", wordcount_delta_path).start()


root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- count: long (nullable = true)



In [47]:
stream.stop()

In [33]:
spark.stop()